In [1]:
import os
import pathlib
import sys
import time

import numpy as np
import pandas as pd
import psutil
from arg_parsing_utils import check_for_missing_args, parse_args
from notebook_init_utils import bandicoot_check, init_notebook

root_dir, in_notebook = init_notebook()

from area_size_shape_utils import measure_3D_area_size_shape
from area_size_shape_utils_gpu import measure_3D_area_size_shape_gpu
from loading_classes import ImageSetLoader, ObjectLoader
from notebook_init_utils import bandicoot_check, init_notebook
from resource_profiling_util import get_mem_and_time_profiling

image_base_dir = bandicoot_check(
    pathlib.Path(os.path.expanduser("~/mnt/bandicoot")).resolve(), root_dir
)

In [2]:
if not in_notebook:
    arguments_dict = parse_args()
    patient = arguments_dict["patient"]
    well_fov = arguments_dict["well_fov"]
    compartment = arguments_dict["compartment"]
    channel = arguments_dict["channel"]
    processor_type = arguments_dict["processor_type"]
    input_subparent_name = arguments_dict["input_subparent_name"]
    mask_subparent_name = arguments_dict["mask_subparent_name"]
    output_features_subparent_name = arguments_dict["output_features_subparent_name"]

else:
    well_fov = "D11-2"
    patient = "NF0016_T1"
    compartment = "Nuclei"
    channel = "DNA"
    processor_type = "CPU"
    input_subparent_name = "zstack_images"
    mask_subparent_name = "segmentation_masks"
    output_features_subparent_name = "extracted_features"

image_set_path = pathlib.Path(
    f"{image_base_dir}/data/{patient}/{input_subparent_name}/{well_fov}/"
)
mask_set_path = pathlib.Path(
    f"{image_base_dir}/data/{patient}/{mask_subparent_name}/{well_fov}/"
)
output_parent_path = pathlib.Path(
    f"{image_base_dir}/data/{patient}/{output_features_subparent_name}/{well_fov}/"
)
output_parent_path.mkdir(parents=True, exist_ok=True)

In [3]:
channel_n_compartment_mapping = {
    "DNA": "405",
    "AGP": "488",
    "ER": "555",
    "Mito": "640",
    "BF": "TRANS",
    "Nuclei": "nuclei_",
    "Cell": "cell_",
    "Cytoplasm": "cytoplasm_",
    "Organoid": "organoid_",
}

In [4]:
start_time = time.time()
# get starting memory (cpu)
start_mem = psutil.Process(os.getpid()).memory_info().rss / 1024**2

In [5]:
image_set_loader = ImageSetLoader(
    image_set_path=image_set_path,
    mask_set_path=mask_set_path,
    anisotropy_spacing=(1, 0.1, 0.1),
    channel_mapping=channel_n_compartment_mapping,
)

In [6]:
object_loader = ObjectLoader(
    image_set_loader.image_set_dict["DNA"],
    image_set_loader.image_set_dict[compartment],
    "DNA",
    compartment,
)


# area, size, shape
if processor_type == "GPU":
    size_shape_dict = measure_3D_area_size_shape_gpu(
        image_set_loader=image_set_loader,
        object_loader=object_loader,
    )
elif processor_type == "CPU":
    size_shape_dict = measure_3D_area_size_shape(
        image_set_loader=image_set_loader,
        object_loader=object_loader,
    )
else:
    raise ValueError(
        f"Processor type {processor_type} is not supported. Use 'CPU' or 'GPU'."
    )

In [7]:
final_df = pd.DataFrame(size_shape_dict)

# prepend compartment and channel to column names
for col in final_df.columns:
    if col not in ["object_id"]:
        final_df[col] = final_df[col].astype(np.float32)
        final_df.rename(
            columns={col: f"Area.Size.Shape_{compartment}_{col}"},
            inplace=True,
        )

final_df.insert(1, "image_set", image_set_loader.image_set_name)

output_file = pathlib.Path(
    output_parent_path
    / f"AreaSizeShape_{compartment}_{processor_type}_features.parquet"
)
final_df.to_parquet(output_file, index=False)
final_df.head()

,object_id,image_set,Area.Size.Shape_Nuclei_VOLUME,Area.Size.Shape_Nuclei_CENTER.X,Area.Size.Shape_Nuclei_CENTER.Y,Area.Size.Shape_Nuclei_CENTER.Z,Area.Size.Shape_Nuclei_BBOX.VOLUME,Area.Size.Shape_Nuclei_MIN.X,Area.Size.Shape_Nuclei_MAX.X,Area.Size.Shape_Nuclei_MIN.Y,Area.Size.Shape_Nuclei_MAX.Y,Area.Size.Shape_Nuclei_MIN.Z,Area.Size.Shape_Nuclei_MAX.Z,Area.Size.Shape_Nuclei_EXTENT,Area.Size.Shape_Nuclei_EULER.NUMBER,Area.Size.Shape_Nuclei_EQUIVALENT.DIAMETER,Area.Size.Shape_Nuclei_SURFACE.AREA
0,6,D11-2,61.0,1227.377075,56.377048,29.000000,90.0,1223.0,1233.0,52.0,61.0,29.0,30.0,0.677778,1.0,4.884016,NaN
1,124,D11-2,64936.0,923.860229,933.460022,12.984092,125939.0,864.0,971.0,884.0,991.0,8.0,19.0,0.515615,1.0,49.868805,385.359161
2,130,D11-2,4636.0,829.319214,964.045532,9.512079,11520.0,781.0,871.0,940.0,1004.0,9.0,11.0,0.402431,1.0,20.687830,15.084225
3,159,D11-2,118796.0,821.394348,1001.115601,8.560684,283200.0,768.0,888.0,939.0,1057.0,0.0,20.0,0.419477,1.0,60.991280,752.229004
4,255,D11-2,199875.0,1013.504822,1007.189575,19.190254,621810.0,933.0,1080.0,932.0,1073.0,2.0,32.0,0.321441,1.0,72.541512,1407.958496


In [8]:
end_mem = psutil.Process(os.getpid()).memory_info().rss / 1024**2
end_time = time.time()
get_mem_and_time_profiling(
    start_mem=start_mem,
    end_mem=end_mem,
    start_time=start_time,
    end_time=end_time,
    feature_type="AreaSizeShape",
    well_fov=well_fov,
    patient_id=patient,
    channel="DNA",
    compartment=compartment,
    CPU_GPU=processor_type,
    output_file_dir=pathlib.Path(
        f"{image_base_dir}/data/{patient}/extracted_features/run_stats/{well_fov}_AreaSizeShape_DNA_{compartment}_{processor_type}.parquet"
    ),
)


        Memory and time profiling for the run:

        Patient ID: NF0016_T1

        Well and FOV: D11-2

        Feature type: AreaSizeShape

        CPU/GPU: CPU")

        Memory usage: 1139.78 MB

        Time:

        --- %s seconds --- % 17.63811683654785

        --- %s minutes --- % 0.2939686139424642

        --- %s hours --- % 0.00489947689904107
    


True